In [1]:
#Installing emot library
!pip install emot

  Obtaining dependency information for emot from https://files.pythonhosted.org/packages/28/b3/3a0f1f66c448fc66aba69a73d6405a104fd973142594ee8bc5a5122e949c/emot-3.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/61.5 kB ? eta -:--:--
   ------ --------------------------------- 10.2/61.5 kB ? eta -:--:--
   ------ --------------------------------- 10.2/61.5 kB ? eta -:--:--
   ------ --------------------------------- 10.2/61.5 kB ? eta -:--:--
   ------ --------------------------------- 10.2/61.5 kB ? eta -:--:--
   ------------------- -------------------- 30.7/61.5 kB 119.1 kB/s eta 0:00:01
   --------------------------------- ------ 51.2/61.5 kB 174.3 kB/s eta 0:00:01
   ---------------------------------------- 61.5/61.5 kB 192.6 kB/s eta 0:00:00


In [2]:
#create sample text data with emoji
text1 = "What are you saying . I am the boss , and why are you so "

In [ ]:
#https://emojijoy.com/
#https://unicode.org/emoji/charts/full-emoji-list.html
#https://apps.timwhitlock.info/emoji/tables/unicode
#Windows+. -> display emoji

In [4]:
import re
from emot.emo_unicode import UNICODE_EMOJI
from emot.emo_unicode import EMOTICONS_EMO

In [6]:
def converting_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","")\
                                           .replace(":","").split()))
    return text
converting_emojis(text1)

'What are you saying . I am the boss , and why are you so '

In [9]:
def emoji_removal(string):
    emoji_unicodes = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols 
                               u"\U0001F680-\U0001F6FF"  # transport 
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"                                
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_unicodes.sub(r'', string)
emoji_removal(text1)

'What are you saying . I am the boss , and why are you so '

In [2]:
import pandas as pd
dataset = pd.read_csv('hate_speech.csv') 
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
dataset.shape

(5242, 3)

In [4]:
dataset.label.value_counts()

label
0    3000
1    2242
Name: count, dtype: int64

In [5]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
    print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [8]:
import re
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = text.lower()  
    return text

In [9]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [10]:
dataset.head(10)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i'm it's so gr...


In [11]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

179

In [12]:
stop = stopwords.words('english')

In [13]:
def gen_freq(text):
    word_list = []
    for tw_words in text.split():
        word_list.extend(tw_words)
    word_freq = pd.Series(word_list).value_counts()
    word_freq = word_freq.drop(stop, errors='ignore')
    return word_freq

In [14]:
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
        else:
            return 0

In [15]:
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
        else:
            return 0

In [16]:
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who', 'where']:
            return 1
        else:
            return 0

In [17]:
def gen_freq(text):
    word_list = []
    for tw_words in text.split():
        word_list.extend(tw_words)
    word_freq = pd.Series(word_list).value_counts()
    word_freq = word_freq.drop(stop, errors='ignore')
    return word_freq

In [21]:
word_freq = gen_freq(dataset.clean_text.str)
rare_100 = word_freq[-100:]
dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))
dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x))
dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x))
dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))
dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))